<a href="https://colab.research.google.com/github/oserikov/data-science-nlp/blob/master/2_%D0%BC%D0%BE%D1%80%D1%84%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip -qq install yargy --progress-bar off
!pip -qq install pymorphy2 --progress-bar off
!pip -qq install -U PyYAML --progress-bar off
!pip -qq install rnnmorph --progress-bar off
!pip -qq install rusenttokenize --progress-bar off


import nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)




import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression

from collections import defaultdict as dd
from operator import itemgetter
from pymorphy2.tokenizers import simple_word_tokenize as word_tokenize
from nltk.corpus import stopwords
from rusenttokenize import ru_sent_tokenize
import string
import pymorphy2
from rnnmorph.predictor import RNNMorphPredictor


import warnings
def warn(*args, **kwargs):
    pass

warnings.warn = warn

## Морфологический анализ

Задачи морфологического анализа:

* Разбор слова — определение нормальной формы (леммы), основы (стема) и грамматических характеристик слова
* Синтез словоформы — генерация словоформы по заданным грамматическим характеристикам из леммы

Морфологический анализ — не самая сильная сторона NLTK.

## POS-tagging

**Частеречная разметка**, или **POS-tagging** _(part of speech tagging)_ —  определение части речи и грамматических характеристик слов в тексте (корпусе) с приписыванием им соответствующих тегов.

Для большинства слов возможно несколько разборов (т.е. несколько разных лемм, несколько разных частей речи и т.п.). Теггер генерирует  все варианты, ранжирует их по вероятности и по умолчанию выдает наиболее вероятный. Выбор одного разбора из нескольких называется **снятием омонимии**, или **дизамбигуацией**.

### Наборы тегов

Существует множество наборов грамматических тегов, или тегсетов:
* НКРЯ
* Mystem
* UPenn
* OpenCorpora (его использует pymorphy2)
* Universal Dependencies
* ...

Есть даже [библиотека](https://github.com/kmike/russian-tagsets) для преобразования тегов из одной системы в другую для русского языка, `russian-tagsets`. Но важно помнить, что преобразования бывают с потерями.

На данный момент стандартом является **Universal Dependencies**. Подробнее про проект можно почитать [вот тут](http://universaldependencies.org/), а про теги — [вот тут](http://universaldependencies.org/u/pos/). Вот список основных (частереных) тегов UD:

* ADJ: adjective
* ADP: adposition
* ADV: adverb
* AUX: auxiliary
* CCONJ: coordinating conjunction
* DET: determiner
* INTJ: interjection
* NOUN: noun
* NUM: numeral
* PART: particle
* PRON: pronoun
* PROPN: proper noun
* PUNCT: punctuation
* SCONJ: subordinating conjunction
* SYM: symbol
* VERB: verb
* X: other



### pymystem3

**pymystem3** — это питоновская обертка для яндексовского морфологичекого анализатора Mystem. Его можно скачать отдельно и использовать из консоли. Может работать с незнакомыми словами (out-of-vocabulary words, OOV).

**pymystem3 не работает в google colaboratory.**

* [Документация Mystem](https://tech.yandex.ru/mystem/doc/index-docpage/)
* [Документация pymystem3](http://pythonhosted.org/pymystem3/)

Инициализируем Mystem c дефолтными параметрами. А вообще параметры есть такие:
* mystem_bin - путь к `mystem`, если их несколько
* grammar_info - нужна ли грамматическая информация или только леммы (по дефолту нужна)
* disambiguation - нужно ли снятие омонимии - дизамбигуация (по дефолту нужна)
* entire_input - нужно ли сохранять в выводе все (пробелы всякие, например), или можно выкинуть (по дефолту оставляется все)

Методы Mystem принимают строку, токенизатор вшит внутри. Можно, конечно, и пословно анализировать, но тогда он не сможет учитывать контекст.

###  pymorphy2

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). Может работать с незнакомыми словами (out-of-vocabulary words, OOV).

[Документация pymorphy2](https://pymorphy2.readthedocs.io/en/latest/)

### rnnmorph
**rnnmorph** &mdash; это морфологический анализатор на нейросетях, занявший первое место на дорожке по морофологическому анализу ["Диалога 2017"](http://www.dialog-21.ru/evaluation/2017/morphology/). 

Предлагает меньшее количество тегов в разборе.

Работает заметно медленнее, чем pymorphy и mystem.

[Документация rnnmorph](https://github.com/IlyaGusev/rnnmorph)

### maru
**maru** — это морфологический анализатор на нейросетях, в документации указаны результаты чуть лучше, чем у rnnmorph на той же задаче.

Тоже медленный

[Документация maru](https://github.com/chomechome/maru)

### Классификация новостей Ленты по темам

In [0]:
! wget -q -O lenta-ru-news-part.csv https://www.dropbox.com/s/ja23c9l1ppo9ix7/lenta-ru-news-part.csv?dl=0

In [0]:
lenta = pd.read_csv('lenta-ru-news-part.csv', usecols=['title', 'text', 'topic'])
lenta["topic_cat"] = lenta.topic.astype('category').cat.codes

def reload_lenta_dataset():
    global lenta
    lenta = pd.read_csv('lenta-ru-news-part.csv', usecols=['title', 'text', 'topic'])
    lenta = lenta.sample(frac=1).reset_index(drop=True).dropna(subset = ['text', 'topic'])
    lenta["topic_cat"] = lenta.topic.astype('category').cat.codes



lenta.head()

,title,text,topic,topic_cat
0,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,3
1,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал...,Спорт,3
2,Названы регионы России с самым дорогим и дешев...,Производитель онлайн-касс «Эвотор» проанализир...,Экономика,4
3,Россию и Украину пригласили на переговоры по газу,Вице-президент Еврокомиссии Марош Шефчович при...,Экономика,4
4,Хоккеист НХЛ забросил шайбу с отрицательного угла,Нападающий клуба «Эдмонтон Ойлерс» Коннор Макд...,Спорт,3


In [0]:
lenta.topic.value_counts()

Экономика          79538
Спорт              64421
Культура           53803
Наука и техника    53136
Бизнес              7399
Name: topic, dtype: int64

In [0]:

def train_test_split(source_col, target_col, source_features_encoder):
    X = source_features_encoder(source_col)
    y = target_col
    
    training_size = len(source_col)*70//100
    X_train, y_train = X[:training_size], y[:training_size]
    X_test, y_test = X[training_size:], y[training_size:]
    
    return X_train, y_train, X_test, y_test


def run_experiment(dataset, dataset_part_size, source_features_encoder):
    
    
    print(len(dataset))
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for i in range(5):
        dataset_shuf = dataset.sample(frac=1)[:dataset_part_size]
        print(len(dataset_shuf))

        X_train, y_train, X_test, y_test = train_test_split(dataset_shuf.text, dataset_shuf.topic_cat, source_features_encoder)

        clf = LogisticRegression()
        clf.fit(X_train, y_train)

        predictions = clf.predict(X_test)

        precision = precision_score(y_test.values, predictions, average='weighted')
        recall = recall_score(y_test.values, predictions, average='weighted')
        f1 = f1_score(y_test.values, predictions, average='weighted')

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        print(f"precision {str(precision)}")
        print(f"recall {str(recall)}")
        print(f"f1-score {str(f1)}")

    print('*' * 10)
    print("mean precision", np.mean(precision_scores))
    print("mean recall", np.mean(recall_scores))
    print("mean f1-score", np.mean(f1_scores))

### наивное решение

In [0]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))

def naive_features_tfidf_encoder(text_col):
    return tfidf_vectorizer.fit_transform(text_col)

In [0]:
run_experiment(lenta, 5000, naive_features_tfidf_encoder)


258297
5000
precision 0.8992486561111459
recall 0.9266666666666666
f1-score 0.9115150924166612
5000
precision 0.9198654085071345
recall 0.9393333333333334
f1-score 0.9282856317362543
5000
precision 0.9126459128746067
recall 0.9366666666666666
f1-score 0.9234252689200926
5000
precision 0.8990482969104359
recall 0.9266666666666666
f1-score 0.911275289156526
5000
precision 0.9133780100337434
recall 0.9366666666666666
f1-score 0.9240829650569145
**********
mean precision 0.9088372568874131
mean recall 0.9332
mean f1-score 0.9197168494572898


**Ого!**

### данные посложнее

In [0]:
!wget -q https://github.com/BobaZooba/HSE-Deep-Learning-in-NLP-Course/blob/master/week_05/data/answers_subsample.csv?raw=true -O data.csv

In [0]:
data = pd.read_csv('data.csv')
data.columns=['topic', 'text']
data["topic_cat"] = data.topic.astype('category').cat.codes
data.head()

,topic,text,topic_cat
0,business,Могут ли в россельхозбанке дать в залог норков...,0
1,law,Может ли срочник перевестись на контракт после...,2
2,business,Продажа недвижимости по ипотеки ? ( арестованы...,0
3,business,"В чем смысл криптовалюты, какая от неё выгода ...",0
4,law,часть 1 статья 158 похитил телефон,2


In [0]:
run_experiment(data, 50000, naive_features_tfidf_encoder)


237779
50000
precision 0.7653119634205529
recall 0.7617333333333334
f1-score 0.759140827826239
50000
precision 0.755805469854021
recall 0.7494666666666666
f1-score 0.7473729674473154
50000
precision 0.7665933673550389
recall 0.7638
f1-score 0.7614770959056998
50000
precision 0.7621465103987121
recall 0.758
f1-score 0.7555892118494607
50000
precision 0.7612214526115553
recall 0.757
f1-score 0.7546073388418562
**********
mean precision 0.762215752727976
mean recall 0.7580000000000001
mean f1-score 0.7556374883741143


### может, нам поможет морфология?


In [0]:
morph_analyzer = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')

tokenization_results = {}

def pymorphy_preprocess_tokenize(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)]
        clean_words = [word for word in clean_words if word]
        clean_words = [word.lower() for word in clean_words if word]
        clean_words = [word for word in clean_words if word not in russian_stopwords]
        
        clean_lemmas = []
        for word in clean_words:
            lemma = tokenization_results.get(word, None) 
            if lemma is None:
                lemma = morph_analyzer.parse(word)[0].normal_form
            tokenization_results[word] = lemma
            clean_lemmas.append(lemma)
        
        text_preprocessed_tokenized.extend(clean_lemmas)

    return text_preprocessed_tokenized

pymorphy_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), tokenizer=pymorphy_preprocess_tokenize)
def pymorphy_features_tfidf_encoder(text_col):
    return pymorphy_tfidf_vectorizer.fit_transform(text_col)

run_experiment(data, 50000, pymorphy_features_tfidf_encoder)

237779
50000
precision 0.7950074806864773
recall 0.7916666666666666
f1-score 0.7906428054929162
50000
precision 0.7945388355749257
recall 0.7915333333333333
f1-score 0.7904031318755138
50000
precision 0.801018428560814
recall 0.7968666666666666
f1-score 0.7963669009195034
50000
precision 0.7967151285353029
recall 0.7928
f1-score 0.7915556436382343
50000
precision 0.8019004270567335
recall 0.7982
f1-score 0.7977549701820409
**********
mean precision 0.7978360600828507
mean recall 0.7942133333333333
mean f1-score 0.7933446904216417


In [0]:
run_experiment(data, 1000, naive_features_tfidf_encoder)

237779
1000
precision 0.5344938124017621
recall 0.42
f1-score 0.3356690838262896
1000
precision 0.46958754813895004
recall 0.42333333333333334
f1-score 0.36455190448325464
1000
precision 0.5504995393403352
recall 0.47333333333333333
f1-score 0.4266368759642257
1000
precision 0.5870619777097482
recall 0.42
f1-score 0.37016441896441904
1000
precision 0.6281292041292041
recall 0.44
f1-score 0.3817159307751757
**********
mean precision 0.553954416344
mean recall 0.43533333333333335
mean f1-score 0.375747642802673


In [0]:
run_experiment(data, 1000, pymorphy_features_tfidf_encoder)

237779
1000
precision 0.7249940801457195
recall 0.4866666666666667
f1-score 0.43055405774160116
1000
precision 0.5905386952229058
recall 0.49
f1-score 0.46024582602948366
1000
precision 0.7150833333333334
recall 0.4533333333333333
f1-score 0.42506535947712415
1000
precision 0.562933736585256
recall 0.4633333333333333
f1-score 0.41780203212311584
1000
precision 0.6767779453661807
recall 0.4766666666666667
f1-score 0.4235553277174007
**********
mean precision 0.6540655581306791
mean recall 0.47400000000000003
mean f1-score 0.43144452061774513


### rnnmorph

In [0]:

predictor = RNNMorphPredictor(language="ru")



def rnnmorph_preprocess_tokenize(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)]
        clean_words = [word for word in clean_words if word]
        clean_words = [word.lower() for word in clean_words if word]
        
        clean_lemmas = [analysis.pos + '_' + analysis.normal_form for analysis in predictor.predict(clean_words)]
        text_preprocessed_tokenized.extend(clean_lemmas)

    return text_preprocessed_tokenized



rnnmorph_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), tokenizer=rnnmorph_preprocess_tokenize)
def rnnmorph_features_tfidf_encoder(text_col):
    return rnnmorph_tfidf_vectorizer.fit_transform(text_col)

run_experiment(data, 1000, rnnmorph_features_tfidf_encoder)

237779
1000
precision 0.5663694675948622
recall 0.48
f1-score 0.4159283093975872
1000
precision 0.5978528310287031
recall 0.49
f1-score 0.41587330534286093
1000
precision 0.622715276131359
recall 0.5666666666666667
f1-score 0.5563000809893157
1000
precision 0.6048095689888859
recall 0.44
f1-score 0.3673116039752533
1000
precision 0.6166377240437757
recall 0.5166666666666667
f1-score 0.5045901070918929
**********
mean precision 0.6016769735575173
mean recall 0.49866666666666665
mean f1-score 0.45200068135938204
